# Web Search 2018 - Tutorial 4: Graph method - Label Propagation
## Contents

1. [Overview](#head1)
  1. [Code Imports](#head11)
2. [Iterative Label Propagation](#head2)
  1. [Dataset - MNIST Digits](#head21)
  2. [Parameters Initialization](#head22)
  3. [Algorithm Implementation](#head23)
  4. [Evaluation](#head24)
  5. [Parameter Tuning](#head25)
3. [Iterative Label Propagation over different Search Spaces](#head3)


## <a name="head1"></a> Overview

Despite the fact that large amounts of data are available in the Web domain, only a small set of such data may be categorized.
The goal of this lab is to understand how one can leverage on a reduced set of categorized data, to enrich sets of uncategorized data. 

Specifically, in this lab you will be using the Label Propagation (LP) algorithm, which consists of a semi-supervised graph approach to annotate uncategorized/unlabelled data starting from a small set of categorized/labelled data.

**Lab objectives:**
* Implement the iterative version of the Label Propagation algorithm and apply it to the MNIST digits dataset
* Evaluate the performance of the Iterative LP algorithm;
* Create multiple graphs, one per feature space (visual and textual), and apply the Iterative LP algorithm.

### <a name="head11"></a> Code Imports

In [1]:
import numpy as np
from numpy.random import shuffle

# <a name="head2"></a> Iterative Label Propagation

Consider a dataset $X=\{x_1, x_2, \ldots, x_L, \ \ x_{L+1}, \ldots, x_N\}$, with $N$ data points, where each $x_i$ consists of some feature representation of document $i$. Given a categories set $C=\{1, 2, \ldots, |C|\}$, it is assumed that the first $L$ data points are labelled with a label $c \in C$, and the remaining ones are unlabelled.

You are asked to implement the Iterative Label Propagation algorithm. Please refer to the "Mining Data Graphs" class (lectured on 29/10), namely slides 28, 29 and 30, for a description of the algorithm steps.

For more information, you can check the original paper: Dengyong Zhou, Olivier Bousquet, Thomas Navin Lal, Jason Weston, Bernhard Schoelkopf. Learning with local and global consistency (2004) http://citeseer.ist.psu.edu/viewdoc/summary?doi=10.1.1.115.3219


## <a name="head21"></a> Dataset - MNIST Digits

Every time you implement an algorithm, it is good practice to debug your implementation on a simple dataset. For this purpose, you should first use the MNIST Digits datasets. This dataset is widely used as a baseline to evaluate machine learning algorithms. You can learn more about it in https://en.wikipedia.org/wiki/MNIST_database.

In [2]:
# Load mnist digits dataset using sklearn. Every digit image is represented as a 8x8 (64) RGB image.
from tokenizer import tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import pandas as pd
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.io import imread
from sklearn.preprocessing import normalize
from skimage import color
from skimage import data, exposure
import random
from numpy.random import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from skimage import img_as_ubyte
import warnings
import os
import time
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
#import cv2

warnings.filterwarnings('ignore')


print(os.getcwd(), "changing to:", os.getcwd()+"/../")

# Change this according to the path where you have the ws_toolkit
ws_toolkit_path = os.getcwd()+"/.."

os.chdir(ws_toolkit_path)
print(os.getcwd())
from ws_toolkit.utils import center_crop_image, k_neighbours, hoc, init_bow

from sklearn import datasets
mnist_digits = datasets.load_digits()
images = mnist_digits.images
print("MNIST digits images shape: {}".format(mnist_digits.images[0].shape))
print("MNIST digits dataset shape: {}".format(mnist_digits.data.shape))
print("MNIST digits categories shape: {} - Categories C: {}".format(mnist_digits.target.shape, set(mnist_digits.target)))

Using TensorFlow backend.


/Users/ZeRafa/Documents/PW/pw_phase2/labs changing to: /Users/ZeRafa/Documents/PW/pw_phase2/labs/../
/Users/ZeRafa/Documents/PW/pw_phase2
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
MNIST digits images shape: (8, 8)
MNIST digits dataset shape: (1797, 64)
MNIST digits categories shape: (1797,) - Categories C: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [3]:
indices = np.arange(len(mnist_digits.data))

# Shuffle the array - Modifies the array inplace 
shuffle(indices)

X = mnist_digits.data[indices]
y_target = mnist_digits.target[indices]

total_images = X.shape[0]

## <a name="head22"></a>  Parameters Initialization

In [4]:
# Let's assume that 20% of the dataset is labeled
labeled_set_size = int(total_images*0.2)

# You should tune these values
alpha = 0.5
num_iterations = 1000
sigma = 1

In [5]:
indices_labeled = indices[:labeled_set_size]
indices_unlabeled = indices[labeled_set_size:]

print("Total images labeled: {} - Total images unlabeled: {}".format(len(indices_labeled), len(indices_unlabeled)))

Total images labeled: 359 - Total images unlabeled: 1438


In [6]:
# Convert labels to a one-hot-encoded vector
from keras.utils import to_categorical

# Keep groundtruth labels
Y_true = to_categorical(y_target)

Y = to_categorical(y_target)
print(Y.shape)

# Remove labels of "unlabeled" data
Y[indices_unlabeled,:] = np.zeros(Y.shape[1])

(1797, 10)


In [7]:
def process_images_keras(_images):
    processedImgs = []
    for img in _images:
        # We are loading each image using Keras image and specifying the target size.
        #image = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

        # Then the function img_to_array converts a PIL (library used by Keras) to a numpy array
        x = img.img_to_array(img)

        # A one dimension is added to the the numpy array (224x224x3) becomes (1x224x224x3)
        x = np.expand_dims(img, axis=0)

        # Apply Keras pre-processing steps specific to VGG16 model
        x = preprocess_input(x)
        processedImgs.append(x)
    return processedImgs

## <a name="head23"></a>  Algorithm Implementation

In [8]:
# Step 1 - Extract features for each image (HoG/CNN/HoC) in X
import numpy as np
from numpy import linalg as LA
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.io import imread
from sklearn.preprocessing import normalize
from skimage import color
from skimage import data, exposure

feature = "hog"
X1 = []
if feature is "hog":
    pixels_per_cell=(2,2)
    orientations=8
    for img in mnist_digits.images:
        hist, hog_img = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, visualize=True, block_norm='L2-Hys')
        hist = np.squeeze(normalize(hist.reshape(1, -1), norm="l2"))
        X1.append(hist)
    X1 = np.array(X)
    print("Shape of feature matrix: {}".format(X.shape))
elif feature is "hoc":
    bins = (2,2,2)
    for img in images:    
        # Change image color space from RGB to HSV. 
        # HSV color space was designed to more closely align with the way human vision perceive color-making attributes
        img_q = img
        img_q = color.rgb2hsv(img)    
        # convert image pixels to [0, 255] range, and to uint8 type
        img_q = img_as_ubyte(img_q)
        # Extract HoC features
        hist, bin_edges = hoc(img_q, bins=bins)    
        # Normalize features
        # We add 1 dimension to comply with scikit-learn API
        hist = np.squeeze(normalize(hist.reshape(1, -1), norm="l2"))    
        X1.append(hist)    
    # Creating a feature matrix for all images
    X1 = np.array(X)
else:
    model = VGG16(weights='imagenet', include_top=True)
    start = time.time()
    start_i = time.time()
    img_list = process_images_keras(images)
    end_i = time.time()
    print("Processed Images finished: {}".format(end_i - start_i))
    #model = ResNet50(weights='imagenet')
    # Convert from list to ndarray
    img_array_list = np.vstack(img_list)
    # Feed all images to the model
    print("No Cached Predictions")
    dataSetPredictions = model.predict(img_array_list)
    end = time.time()
    print("Model Predictions finished: {}".format(end - start))
    #print("Resulting shape of the network output: {}".format(preds.shape))
    concepts = decode_predictions(dataSetPredictions, top=5)
    # Experiment with this parameter
    k = 5
    # Get the top K most probable concepts per image
    sorted_concepts =  np.argsort(dataSetPredictions, axis=1)[:,::-1][:,:k]
    data_tags = concepts
    mlb = MultiLabelBinarizer(classes=range(0,1000))
    tags_bow = mlb.fit_transform(sorted_concepts)
    X1 = tags_bow
    

# Step 2 - Initialize matrix F
# Normalize Y_hidden such that each row sums to 1.
Y_hidden = normalize(Y, axis = 1, norm="l1")
F = Y_hidden
print(F[indices_labeled[0],:])

# Step 3 - Compute matrix W
from sklearn.metrics.pairwise import euclidean_distances
M = euclidean_distances(X1, X1)
#sigma = np.std(M)
W = np.exp(-1*M/(2*sigma**2))

# Step 4 - Normalize W
D = np.zeros(W.shape)
np.fill_diagonal(D, W.sum(axis=0))

D12 = np.zeros(D.shape)
from numpy.linalg import inv
D12 = inv(np.sqrt(D))

S = np.dot(D12, W)
S = np.dot(S, D12)

# Step 5 - Perform the F update step num_iterations steps
for i in range(1, num_iterations):
    T1 = alpha * S
    T1 = np.dot(T1,F)
    T2 = (1-alpha) * Y_hidden
    F = T1 + T2
    F = normalize(F, axis = 1, norm="l1")
    print(F[indices_unlabeled[0],:])
F = np.argmax(F, axis=1)
Y = to_categorical(F)

Shape of feature matrix: (1797, 64)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.55336451e-07 1.14384195e-04 5.93322595e-08 3.30663182e-07
 9.99872373e-01 1.72345328e-06 4.56195512e-06 2.77591329e-06
 3.25938106e-06 2.76419975e-07]
[2.68826612e-07 1.26236057e-04 6.62955130e-08 3.42773072e-07
 9.99860034e-01 1.79358392e-06 4.73406540e-06 2.86866257e-06
 3.37150414e-06 2.84722029e-07]
[2.82374762e-07 1.38077752e-04 7.32648811e-08 3.54923656e-07
 9.99847703e-01 1.86408103e-06 4.90641579e-06 2.96159273e-06
 3.48368612e-06 2.93046995e-07]
[2.95972726e-07 1.49906971e-04 8.02385160e-08 3.67108854e-07
 9.99835384e-01 1.93490123e-06 5.07887831e-06 3.05459773e-06
 3.59588257e-06 3.01392137e-07]
[3.09616360e-07 1.61722573e-04 8.72154829e-08 3.79325425e-07
 9.99823078e-01 2.00602101e-06 5.25138852e-06 3.14762458e-06
 3.70807099e-06 3.09756032e-07]
[3.23303532e-07 1.73523999e-04 9.41953019e-08 3.91571541e-07
 9.99810787e-01 2.07742685e-06 5.42391368e-06 3.24064678e-06
 3.82023980e-06 3.18137912e-07]
[3.37033

## <a name="head24"></a>  Evaluation

Evaluate the results of each run of the Iterative LP.

In [9]:
# The function classification_report computes and prints a set of commonly used metrics.
# docs: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report
from sklearn.metrics import classification_report
np.set_printoptions(threshold=np.nan)

# Get the predictions of the unlabeled documents
Y_pred = Y[indices_unlabeled, :]
#print(Y_pred)

# Get the corresponding groundtruth
y_gt = Y_true[indices_unlabeled, :]
#print(y_gt)

print(classification_report(y_gt, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       138
           1       0.93      1.00      0.96       146
           2       0.98      1.00      0.99       128
           3       0.95      0.99      0.97       148
           4       0.99      0.99      0.99       147
           5       0.98      0.97      0.98       145
           6       0.99      0.99      0.99       137
           7       0.99      0.99      0.99       146
           8       0.99      0.87      0.93       151
           9       0.96      0.95      0.95       152

   micro avg       0.97      0.97      0.97      1438
   macro avg       0.98      0.98      0.97      1438
weighted avg       0.97      0.97      0.97      1438
 samples avg       0.97      0.97      0.97      1438



# <a name="head3"></a> Iterative Label Propagation over different Search Spaces

Re-run your Iterative LP implementation, for each of the previously implemented search spaces (HoG, CNN and HoC).

Evaluate and discuss the obtained results.